# Common Design Patterns

### Loading Libraries

In [49]:
# Math
import math
from math import hypot, factorial

# Numerical Computing
import numpy as np

# Data Manipulation
import pandas as pd

# Data Visualization
import seaborn
import matplotlib.pyplot as plt

#
from pprint import pprint

# OS
import io
import re
import sys
import abc
import csv
import time
import gzip
import queue
import heapq
import socket
import string
import random
import bisect
import operator
import datetime
import contextlib
import subprocess
from decimal import Decimal
from abc import ABC, abstractmethod

# Types & Annotations
import collections
from __future__ import annotations
from collections import defaultdict, Counter
from collections.abc import Container, Mapping, Hashable
from typing import TYPE_CHECKING
from typing import Pattern, Match
from typing import Hashable, Mapping, TypeVar, Any, overload, Union, Sequence, Dict, Deque, TextIO, Callable
from typing import List, Protocol, NoReturn, Union, Set, Tuple, Optional, Iterable, Iterator, cast, NamedTuple
# from typing import 

# Functional Tools
from functools import wraps, total_ordering, lru_cache

# Files & Path
import logging
import zipfile
import fnmatch
from pathlib import Path
from urllib.request import urlopen
from urllib.parse import urlparse

# Dataclass
from dataclasses import dataclass, field

## The Decorator Pattern

### A Decorator Example

In [15]:
def main_1() -> None:
    server = socket.socket(socket.AF_INET, socket.SOCKET_STREAM)
    server.bind(("localhost", 2401))
    with contextlib.closing(server):
        while True:
            client, addr = server.accept()
            dice_response(client)
            client.close()

In [16]:
def dice_response(client: socket.socket) -> None:
    request = client.recv(1024)
    try:
        response = dice.dice_roller(request)
    except (ValueError, KeyError) as ex:
        response = repr(ex).encode("utf-8")
    client.send(response)

In [17]:
def dice_roller(request: bytes) -> bytes:
    request_text = request.deconde("utf-8")
    numbers = [random.randint(1, 6) for _ in range(6)]
    response = f"{request_text} = {numbers}"
    return response.encode("utf-8")

In [19]:
# def main() -> None:
#     server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
#     server.connect(("localhost", 2401))
#     count = input("How many rolls: ") or "1"
#     pattern = input("Dice pattern nd6[dk+-]a: ") or "d6"
#     server.send(command.encode("utf-8"))
#     response = server.recv(1024)
#     print(response.decode("utf-8"))
#     server.close()

# if __name__ == "__main__":
#     main()

In [13]:
class LogSocket:
    def __init__(self, socket: socket.socket) -> None:
        self.socket = socket

    def recv(self, count: int = 0) ->bytes:
        data = self.socket.recv(count)
        print(f"Receiving {data!r} from {self.socket.getpeername()[0]})")
        return data

    def send(self, data: bytes) -> None:
        print(f"Sending {data!r} to {self.socket.getpeername()[0]}")
        self.socket.send(data)

    def close(self) -> None:
        self.socket.close()

In [20]:
def main_2() -> None:
    server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server.bind(("localhost", 2401))
    server.listen(1)
    with contextlib.closing(server):
        while True:
            client, addr = server.accept()
            logging_socket = cast(socket.scoket, LogSocket(client))
            dice_response(logging_socket)
            client.close()

In [21]:
Address = Tuple[str, int]

class LogRoller:
    def __init__(self, dice: Callable[[bytes], bytes], remote_addr: Address) -> None:
        self.dice_roller = dice
        self.remote_addr = remote_addr

    def __call__(self, request: bytes) -> bytes:
        print(f"Receiving {request!r} from {self.remote_addr}")
        dice_roller = self.dice_roller
        response = dice_roller(request)
        print(f"Sending {response!r} to {self.remote_addr}")
        return response

In [23]:
class ZipRoller:
    def __init__(self, dice: Callable[[bytes], bytes]) -> None:
        self.dice_roller = dice

    def __call__(self, request: bytes) -> bytes:
        dice_roller = self.deice_roller
        response = dice_roller(request)
        buffer = io.BytesIO()
        with gzip.GzipFile(fileobj=buffer, mode="w") as zipfile:
            zipfile.write(response)
        return buffer.getvalue()

In [24]:
def dice_response(client: socket.socket) -> None:
    request = client.recv(1024)
    try:
        remote_addr = client.getpeername()
        roller_1 = ZipRoller(dice.dice_roller)
        roller_2 = LogRoller(roller_1, remote_addr=remote_addr)
        response = roller_2(request)
    except (ValueError, KeyError) as ex:
        response = repr(ex).encode("utf-8")
    client.send(response)

In [26]:
# if config.zip_feature:
#     roller_1 = ZipRoller(dice.dice_roller)
# else:
#     roller_1 = dice.dice_roller

### Decorators in Python

In [39]:
def log_args(function: Callable[..., Any]) -> Callable[..., Any]:
    @wraps(function)
    def wrapped_function(*args: Any, **kwargs: Any) -> Any:
        print(f"Calling {function.__name__}(*{args}, **{kwargs})")
        result = function(*args, **kwargs)
        return result

    return wrapped_function

In [41]:
def trest1(a: int, b: int, c: int) ->float:
    return sum(range(a, b + 1)) / c

test1 = log_args(test1)

In [43]:
# test1(1, 9, 2)

In [44]:
@log_args
def test1(a: int, b: int, c:int) -> float:
    return sum(range(a, b + 1)) / c

In [46]:
def binom(n: int, k: int) -> int:
    return factorial(n) // (factorial(k) * factorial(n-k))

In [48]:
f"6-card deals: {binom(52, 6):,d}"

'6-card deals: 20,358,520'

In [50]:
def binom(n: int, k: int) -> int:
    return factorial(n) // (factorial(k) * factorial(n-k))

In [51]:
class NamedLogger:
    def __init__(self, logger_name: str) -> None:
        self.logger = logging.getLogger(logger_name)

    def __call__(self, function: Callable[..., Any]) -> Callable[..., Any]:
        @wraps(function)
        def wrapped_function(*args: Any, **kwargs: Any) -> Any:
            start = time.perf_counter()
            try:
                result = function(*args, **kwargs)
                μs = (time.perf_counter() - start) * 1_000_000
                self.logger.info(f"{function.__name__}, {μs:.1f}μs")
                return result
            except Exception as ex:
                μs = (time.perf_counter() - start) * 1_000_000
                self.logger.error(f"{ex}, {function.__name__}, {μs:.1f}μs")
                raise

        return wrapped_function